In [20]:
import sparknlp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
import nltk

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
import string
import re

#Import required pyspark libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import udf
from pyspark.sql.functions import col,isnan, when, count
from pyspark.sql.functions import regexp_replace, trim, col, lower

from pyspark.ml import Pipeline

#Import required sparknlp libraries
import sparknlp

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
import os
import sys

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1660508703299_0001,pyspark,idle,Link,Link,None,✔


In [25]:
import nltk
from nltk.corpus import stopwords

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
stop_wrd = list( stopwords.words('english'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark NLP version:  4.0.2
Apache Spark version:  3.2.0-amzn-0

In [28]:
spark = SparkSession.builder.appName('sentiment_analysis').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Reading data from S3 to a DataFrame
reviews_df = spark.read.json('s3://mskindledata/Kindle_Store_5.json')
reviews_df.show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+--------------+-------------------+-----------------+-----------------+--------------+--------+----+
|asin      |image|overall|reviewText                                                                                                                                                                                                                                 |reviewTime |reviewerID    |reviewerName       |style            |summary          |unixReviewTime|verified|vote|
+----------+-----+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
reviews_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[asin: string, image: array<string>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, style: struct<Format::string>, summary: string, unixReviewTime: bigint, verified: boolean, vote: string]

In [31]:
reviews_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Format:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)

In [32]:
reviews_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2222983

In [33]:
reviews_df = reviews_df.dropDuplicates(['reviewTime','reviewerID','asin', 'reviewerName', 'reviewText', 'summary'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
reviews_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2220617

In [35]:
## Dropping null values in columns reviewtext, summary
df = reviews_df.dropna(subset=['reviewText', 'summary'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2218487

In [37]:
columns_to_drop = ['asin','image', 'reviewerID', 'reviewerName', 'unixReviewTime', 'vote', 'verified', 'unixReviewTime', 'style', 'reviewTime']
df = df.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df.printSchema()
#find number of null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)

In [39]:
df.show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2218487

In [41]:
df = df.select("*",concat(col("reviewText"), lit(" "), col("summary")).alias("reviewtext_summary"))
columns_drop =['reviewText', 'summary']
df = df.drop(*columns_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df.show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [43]:
def label_data(x):
    if x==5 or x==4:
        return 'positive'
    if x==3:
        return 'neutral'
    else:
        return 'negative'

labeldata_udf = udf(lambda x:label_data(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df = df.withColumn('labels', labeldata_udf('overall'))
df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [45]:

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()
#Dropping Null 
df = df.dropna()
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------+
|overall|reviewtext_summary|labels|
+-------+------------------+------+
|      0|                 0|     0|
+-------+------------------+------+

2218487

# Data Cleaning

In [46]:
#Data Cleaning

from pyspark.sql.functions import udf
from pyspark.sql.types import *
##Convert to lower case
def convert_to_lower(text):
    return text.lower()

# Remove non-ascii characters
def remove_non_ascii(text):
    stripped = (char for char in text if 0 < ord(char) < 127)
    return ''.join(stripped)

## Expand abbrevations
def expand_abbrevation(text):
    text = re.sub(r'\bim\b', 'i am', text)
    text = re.sub(r'\blol\b', 'laugh out loud', text)
    text = re.sub(r'\bthats\b', 'that is', text)
    text = re.sub(r'\bive\b', 'i have', text)
    text = re.sub(r'\bya\b', 'yeah', text)
    text = re.sub(r'\bcant\b', 'can not', text)
    text = re.sub(r'\bdont\b', 'do not', text)
    text = re.sub(r'\bwont\b', 'will not', text)
    text = re.sub(r'\bid\b', 'i would', text)
    text = re.sub(r'\bwth\b', 'what the hell', text)
    text = re.sub(r'\br\b', 'are', text)
    text = re.sub(r'\bu\b', 'you', text)
    text = re.sub(r'\bk\b', 'OK', text)
    text = re.sub(r'\bsux\b', 'sucks', text)
    text = re.sub(r'\bno+\b', 'no', text)
    text = re.sub(r'\bcoo+\b', 'cool', text)
    text = re.sub(r'rt\b', '', text)
    text = text.strip()
    return text

##Remove some features that are not required
def remove_features(text):
# compile regex
    re_url          = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    re_punctuation  = re.compile('[%s]' % re.escape(string.punctuation))
    re_numeric      = re.compile('(\\d+)')
    re_mention      = re.compile('@(\w+)')
    re_hashtags     = re.compile('#')
    re_alphanumeric = re.compile("^[a-z0-9_.]+$")
# remove url 
    text = re_url.sub(' ', text)
# remove punctuations
    text = re_punctuation.sub(' ', text)
# remove @mentions
    text = re_mention.sub(' ', text)
# remove hashtags
    text = re_hashtags.sub(' ', text)
# remove numeric 'words'
    text = re_numeric.sub(' ', text)
# remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in text.split():
        if list_pos == 0:
            if re_alphanumeric.match(word):
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if re_alphanumeric.match(word):
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
# remove unwanted space, *.split() will automatically split on
# whitespace and discard duplicates, the " ".join() joins the
# resulting list into one string.
    return " ".join(cleaned_str.split())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
convert_to_lower_udf    = udf(convert_to_lower,StringType())
remove_non_ascii_udf    = udf(remove_non_ascii, StringType())
expand_abbrevation_udf  = udf(expand_abbrevation, StringType())
remove_features_udf     = udf(remove_features, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df = df.withColumn("reviewtext_summary",convert_to_lower_udf(df["reviewtext_summary"]))
df = df.withColumn("reviewtext_summary",remove_non_ascii_udf(df["reviewtext_summary"]))
df = df.withColumn("reviewtext_summary",expand_abbrevation_udf(df["reviewtext_summary"]))
df = df.withColumn("reviewtext_summary",remove_features_udf(df["reviewtext_summary"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df.show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|overall|reviewtext_summary                                                                                                                                                                                                                                                                                                                                                                                                                               

# Data Pre-processing

In [50]:
assembler = DocumentAssembler().setInputCol('reviewtext_summary').setOutputCol('document')

doc_df = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
doc_df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
doc_df = sentence_detector.transform(doc_df)
doc_df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o370.getParam.
: java.util.NoSuchElementException: Param customBoundsStrategy does not exist.
	at org.apache.spark.ml.param.Params.$anonfun$getParam$2(params.scala:705)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.ml.param.Params.getParam(params.scala:705)
	at org.apache.spark.ml.param.Params.getParam$(params.scala:703)
	at org.apache.spark.ml.PipelineStage.getParam(Pipeline.scala:41)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.jav

In [53]:
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')#.setTargetPattern('/\b(\?You were looking)\b/')#.setExceptionsPath('/content/MyDrive/SparkNLP/entities.txt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
spell_chk = NorvigSweetingModel().pretrained().setInputCols(['token']).setOutputCol('corrected')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[OK!]

In [55]:
stemmer = Stemmer().setInputCols(["token"]).setOutputCol("stem")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
lemmatizer = LemmatizerModel().pretrained().setInputCols(['stem']).setOutputCol('lemma')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [57]:
normalizer = Normalizer().setInputCols(['lemma']).setOutputCol('normalized').setLowercase(True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
stop_wrd = list( stopwords.words('english'))

stop_words_cleaner = StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False).setStopWords(stop_wrd)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:

finisher = Finisher().setInputCols(['cleanTokens']).setOutputCols(['cleanTokens']).setOutputAsArray(True).setCleanAnnotations(True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:

pipeline_embd = Pipeline().setStages([ assembler, tokenizer, spell_chk ,stemmer, lemmatizer, normalizer,stop_words_cleaner,finisher ]) 
model_trans_tfIdf_2 = pipeline_embd.fit(doc_df) 
model_trans_tfIdf_2 = model_trans_tfIdf_2.transform(doc_df) 
model_trans_tfIdf_2.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [62]:
model_trans_tfIdf_2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------------------+
|overall|  reviewtext_summary|  labels|         cleanTokens|
+-------+--------------------+--------+--------------------+
|    5.0|i was not sure if...|positive|[wa, sure, would,...|
|    4.0|this was an amazi...|positive|[thi, wa, amaz, b...|
|    5.0|i always loved he...|positive|[alwai, love, boo...|
|    5.0|my kind of book a...|positive|[kind, book, like...|
|    4.0|this is a great r...|positive|[thi, great, read...|
|    5.0|what a great read...|positive|[great, read, mes...|
|    5.0|a wonderful new s...|positive|[wonder, new, ser...|
|    4.0|it was a good sto...|positive|[wa, good, stori,...|
|    5.0|shattered ink is ...|positive|[shatter, ink, an...|
|    5.0|mccarthys of gans...|positive|[mccarthi, ganset...|
|    5.0|very good plot gr...|positive|[veri, good, plot...|
|    2.0|shame on you auth...|negative|[shame, author, p...|
|    5.0|i loved this book...|positive|[love, thi, book,...|
|    4.0|but a solid eno

# Data Sampling

In [63]:
model_trans_tfIdf_2.groupBy('labels').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+
|  labels|  count|
+--------+-------+
|positive|1906045|
| neutral| 197620|
|negative| 114822|
+--------+-------+

In [69]:
sampled = model_trans_tfIdf_2.sampleBy("labels", fractions={'positive': 0.07, 'neutral': 0.7, 'negative': 1}, seed=0)
sampled.groupBy('labels').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+
|  labels| count|
+--------+------+
|positive|133605|
| neutral|138673|
|negative|114822|
+--------+------+

In [70]:
sampled.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

386656

# Feature Extraction 

In [71]:
## Count Vectorizer TF*IDF
from pyspark.ml.feature import CountVectorizer, IDF

count_vectorizer = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel = count_vectorizer.fit(model_trans_tfIdf_2)
idf = IDF(inputCol='tf', outputCol='idf')

# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline = Pipeline().setStages([count_vectorizer, idf])
bow_pipeline = bow_pipeline.fit(model_trans_tfIdf_2)

bows = bow_pipeline.transform(model_trans_tfIdf_2)
bows.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------------------+--------------------+--------------------+
|overall|  reviewtext_summary|  labels|         cleanTokens|                  tf|                 idf|
+-------+--------------------+--------+--------------------+--------------------+--------------------+
|    5.0|got hooked from t...|positive|[get, hook, begin...|(46803,[1,3,5,10,...|(46803,[1,3,5,10,...|
|    5.0|awesome read fog ...|positive|[awesom, read, fo...|(46803,[3,172,449...|(46803,[3,172,449...|
|    1.0|possible spoilers...|negative|[possibl, spoiler...|(46803,[0,4,6,7,1...|(46803,[0,4,6,7,1...|
|    4.0|for a yeah na thi...|positive|[yeah, na, thi, w...|(46803,[0,1,2,6,9...|(46803,[0,1,2,6,9...|
|    5.0|this is a truly b...|positive|[thi, truli, beau...|(46803,[0,3,4,5,6...|(46803,[0,3,4,5,6...|
|    4.0|thoroughly enjoya...|positive|[thoroughli, enjo...|(46803,[0,3,4,6,1...|(46803,[0,3,4,6,1...|
|    5.0|loved this series...|positive|[love, thi, seri,...|(46803,[0,5,1

In [72]:
from pyspark.ml.feature import HashingTF

hashtf = HashingTF(numFeatures=pow(2, 16), inputCol='cleanTokens', outputCol='Hashingtf')

hashing_pipeline = Pipeline().setStages([hashtf])
hashing_pipeline = hashing_pipeline.fit(model_trans_tfIdf_2)

hashingtf = hashing_pipeline.transform(model_trans_tfIdf_2)
hashingtf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------------------+--------------------+
|overall|  reviewtext_summary|  labels|         cleanTokens|           Hashingtf|
+-------+--------------------+--------+--------------------+--------------------+
|    5.0|words cannot desc...|positive|[word, cannot, de...|(65536,[88,7772,1...|
|    4.0|very enjoyable re...|positive|[veri, enjoy, rea...|(65536,[2701,2987...|
|    2.0|this book had goo...|negative|[thi, book, good,...|(65536,[2517,3920...|
|    5.0|i know that other...|positive|[know, express, o...|(65536,[12,44,158...|
|    5.0|as always shane t...|positive|[alwai, shane, ta...|(65536,[24476,276...|
|    5.0|i have to admit i...|positive|[admit, love, thi...|(65536,[44,377,89...|
|    5.0|into uncharted se...|positive|[unchart, sea, th...|(65536,[1473,2126...|
|    4.0|i really enjoyed ...|positive|[realli, enjoi, t...|(65536,[637,716,1...|
|    4.0|i enjoyed reading...|positive|[enjoi, read, dif...|(65536,[6747,6783...|
|    2.0|unintel

In [73]:
from pyspark.ml.feature import StringIndexer
label_stringIdx = StringIndexer(inputCol = "labels", outputCol = "label" )
pipeline = Pipeline(stages=[ label_stringIdx])
pipelineFit = pipeline.fit(hashingtf)
dataset = pipelineFit.transform(hashingtf)
selectedCols = ['cleanTokens','Hashingtf','label']
dataset = dataset.select(selectedCols)
dataset.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|           Hashingtf|label|
+--------------------+--------------------+-----+
|[word, cannot, de...|(65536,[88,7772,1...|  0.0|
|[veri, enjoy, rea...|(65536,[2701,2987...|  0.0|
|[thi, book, good,...|(65536,[2517,3920...|  2.0|
|[know, express, o...|(65536,[12,44,158...|  0.0|
|[alwai, shane, ta...|(65536,[24476,276...|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows

In [74]:
from pyspark.ml.feature import StringIndexer
label_stringIdx = StringIndexer(inputCol = "labels", outputCol = "label" )
pipeline = Pipeline(stages=[ label_stringIdx])
pipelineFit = pipeline.fit(bows)
dataset1 = pipelineFit.transform(bows)
selectedCols = ['cleanTokens','idf','label']
dataset1 = dataset1.select(selectedCols)
dataset1.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|                 idf|label|
+--------------------+--------------------+-----+
|[veri, cute, stor...|(46803,[3,4,6,19,...|  0.0|
|[charact, ar, wel...|(46803,[0,1,3,7,1...|  0.0|
|[good, read, five...|(46803,[3,6,35,11...|  0.0|
|[love, novella, l...|(46803,[5,7,9,10,...|  0.0|
|[read, thi, sho, ...|(46803,[0,2,3,4,7...|  1.0|
+--------------------+--------------------+-----+
only showing top 5 rows

# Feature Extraction - Sampling

In [75]:
## Count Vectorizer TF*IDF - with sampled data

count_vectorizer_s = CountVectorizer(inputCol='cleanTokens', outputCol='tf', minDF=10)
cvmodel_sample = count_vectorizer_s.fit(sampled)
idf_sample = IDF(inputCol='tf', outputCol='idf')

# finisher_idf = Finisher().setInputCols(['features']).setOutputCols(['tfidf']) finisher is for sparkNLP not spark ML
bow_pipeline_sample = Pipeline().setStages([count_vectorizer_s, idf])
bow_pipeline_sample = bow_pipeline_sample.fit(sampled)

bows_sample = bow_pipeline_sample.transform(sampled)
bows_sample.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------------------+--------------------+--------------------+
|overall|  reviewtext_summary|  labels|         cleanTokens|                  tf|                 idf|
+-------+--------------------+--------+--------------------+--------------------+--------------------+
|    2.0|this book had goo...|negative|[thi, book, good,...|(20986,[0,1,2,3,4...|(20986,[0,1,2,3,4...|
|    2.0|unintelligibility...|negative|[unintellig, pose...|(20986,[3106,9943...|(20986,[3106,9943...|
|    3.0|the description s...| neutral|[descript, state,...|(20986,[0,1,2,3,4...|(20986,[0,1,2,3,4...|
|    5.0|this book has all...|positive|[thi, book, ha, e...|(20986,[0,1,7,8,1...|(20986,[0,1,7,8,1...|
|    4.0|i started reading...|positive|[start, read, rec...|(20986,[4,10,21,8...|(20986,[4,10,21,8...|
|    3.0|i had already fel...| neutral|[alreadi, feel, l...|(20986,[0,1,2,5,9...|(20986,[0,1,2,5,9...|
|    1.0|was confused when...|negative|[wa, confus, end,...|(20986,[2,24,

In [76]:
from pyspark.ml.feature import HashingTF

hashtf_s = HashingTF(numFeatures=pow(2, 16), inputCol='cleanTokens', outputCol='Hashingtf')

hashing_pipeline_sample = Pipeline().setStages([hashtf_s])
hashing_pipeline_sample = hashing_pipeline_sample.fit(sampled)

hashingtf_sample = hashing_pipeline_sample.transform(sampled)
hashingtf_sample.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------+--------------------+--------------------+
|overall|  reviewtext_summary|  labels|         cleanTokens|           Hashingtf|
+-------+--------------------+--------+--------------------+--------------------+
|    1.0|i was anticipatin...|negative|[wa, anticip, thi...|(65536,[3920,4774...|
|    4.0|holly and chase a...|positive|[holli, chase, ar...|(65536,[44,1198,1...|
|    5.0|unique story this...|positive|[uniqu, stori, th...|(65536,[2206,2701...|
|    3.0|this book was not...| neutral|[thi, book, wa, g...|(65536,[12,1887,6...|
|    1.0|someone on goodre...|negative|[someon, goodread...|(65536,[891,2517,...|
|    3.0|cute holiday sho ...| neutral|[cute, holidai, s...|(65536,[2171,2306...|
|    5.0|this is a feel go...|positive|[thi, feel, good,...|(65536,[16888,331...|
|    4.0|if you re looking...|positive|[look, quick, esc...|(65536,[2,44,535,...|
|    5.0|you have done it ...|positive|[sir, hot, steami...|(65536,[145,15269...|
|    1.0|the wri

In [77]:
from pyspark.ml.feature import StringIndexer
label_stringIdx_s = StringIndexer(inputCol = "labels", outputCol = "label" )
pipeline_s = Pipeline(stages=[ label_stringIdx_s])
pipelineFit_s = pipeline.fit(hashingtf_sample)
dataset_sample = pipelineFit_s.transform(hashingtf_sample)
selectedCols = ['cleanTokens','Hashingtf','label']
dataset_sample = dataset_sample.select(selectedCols)
dataset_sample.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|           Hashingtf|label|
+--------------------+--------------------+-----+
|[bui, hi, next, l...|(65536,[1198,2517...|  2.0|
|[thi, nice, littl...|(65536,[546,1791,...|  1.0|
|[dakota, give, re...|(65536,[2171,2692...|  1.0|
|[thi, book, might...|(65536,[5451,5505...|  0.0|
|[never, deal, adm...|(65536,[2692,6783...|  1.0|
+--------------------+--------------------+-----+
only showing top 5 rows

In [78]:
from pyspark.ml.feature import StringIndexer
label_stringIdx_s = StringIndexer(inputCol = "labels", outputCol = "label" )
pipeline_s = Pipeline(stages=[ label_stringIdx_s])
pipelineFit_s = pipeline.fit(bows_sample)
dataset1_sample = pipelineFit.transform(bows_sample)
selectedCols = ['cleanTokens','idf','label']
dataset1_sample = dataset1_sample.select(selectedCols)
dataset1_sample.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|                 idf|label|
+--------------------+--------------------+-----+
|[thi, book, good,...|(20986,[0,1,2,3,4...|  2.0|
|[unintellig, pose...|(20986,[3106,9943...|  2.0|
|[descript, state,...|(20986,[0,1,2,3,4...|  1.0|
|[thi, book, ha, e...|(20986,[0,1,7,8,1...|  0.0|
|[start, read, rec...|(20986,[4,10,21,8...|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows

#  Split the dataset

In [79]:

(trainingData, testData) = dataset.select('cleanTokens','Hashingtf','label').randomSplit([0.7, 0.3], seed = 100)
trainingData.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|           Hashingtf|label|
+--------------------+--------------------+-----+
|                  []|       (65536,[],[])|  1.0|
|                  []|       (65536,[],[])|  1.0|
|[aaa, entir, seri...|(65536,[12710,167...|  0.0|
|[aaahhh, joi, ret...|(65536,[506,2530,...|  0.0|
|[aah, thi, fair, ...|(65536,[3920,6783...|  0.0|
|[aaron, clarkson,...|(65536,[44,617,11...|  0.0|
|[aaron, clarkson,...|(65536,[44,231,26...|  0.0|
|[aaron, ha, wai, ...|(65536,[2517,3145...|  0.0|
|[aaron, major, se...|(65536,[269,1061,...|  0.0|
|[ab, connor, pa, ...|(65536,[44,330,40...|  0.0|
+--------------------+--------------------+-----+
only showing top 10 rows

In [80]:

(trainingData1, testData1) = dataset1.select('cleanTokens','idf','label').randomSplit([0.7, 0.3], seed = 100)
trainingData1.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|                 idf|label|
+--------------------+--------------------+-----+
|                  []|       (46803,[],[])|  1.0|
|                  []|       (46803,[],[])|  1.0|
|[aaa, entir, seri...|(46803,[13,35,113...|  0.0|
|[aaahhh, joi, ret...|(46803,[0,1,2,3,4...|  0.0|
|[aah, thi, fair, ...|(46803,[0,5,26,29...|  0.0|
|[aaron, clarkson,...|(46803,[5,7,8,10,...|  0.0|
|[aaron, clarkson,...|(46803,[1,2,5,6,7...|  0.0|
|[aaron, ha, wai, ...|(46803,[2,4,8,11,...|  0.0|
|[aaron, major, se...|(46803,[0,1,2,8,1...|  0.0|
|[ab, connor, pa, ...|(46803,[5,6,7,8,1...|  0.0|
+--------------------+--------------------+-----+
only showing top 10 rows

In [81]:

(trainingData_s, testData_s) = dataset_sample.select('cleanTokens','Hashingtf','label').randomSplit([0.7, 0.3], seed = 100)
trainingData_s.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|           Hashingtf|label|
+--------------------+--------------------+-----+
|                  []|       (65536,[],[])|  0.0|
|[aaron, clarkson,...|(65536,[44,617,11...|  1.0|
|[abandon, read, s...|(65536,[16706,397...|  2.0|
|[abigail, darling...|(65536,[2,44,361,...|  0.0|
|[abigail, indepen...|(65536,[44,756,15...|  1.0|
|[abl, complet, gr...|(65536,[2381,3924...|  0.0|
|[abl, read, whole...|(65536,[9645,1122...|  2.0|
|[absolut, ador, f...|(65536,[88,1433,1...|  2.0|
|[absolut, ador, f...|(65536,[2692,6083...|  1.0|
|[absolut, amaz, s...|(65536,[2692,4992...|  1.0|
+--------------------+--------------------+-----+
only showing top 10 rows

In [82]:

(trainingData1_s, testData1_s) = dataset1_sample.select('cleanTokens','idf','label').randomSplit([0.7, 0.3], seed = 100)
trainingData1_s.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+
|         cleanTokens|                 idf|label|
+--------------------+--------------------+-----+
|                  []|       (20986,[],[])|  1.0|
|[aaa, entir, seri...|(20986,[15,33,233...|  0.0|
|[aaron, pilan, li...|(20986,[0,1,2,3,4...|  0.0|
|[abandon, thi, bo...|(20986,[0,1,2,3,4...|  2.0|
|[abbei, foxx, new...|(20986,[0,1,2,3,4...|  0.0|
|[abigail, conner,...|(20986,[0,2,4,7,8...|  0.0|
|[abigail, darling...|(20986,[0,1,2,5,6...|  1.0|
|[abl, read, whole...|(20986,[0,1,3,4,3...|  2.0|
|[abort, privileg,...|(20986,[0,2,3,4,5...|  1.0|
|[absolut, connect...|(20986,[0,1,2,4,1...|  2.0|
+--------------------+--------------------+-----+
only showing top 10 rows

# Classification  Models

## Logistic Regression

In [83]:
from pyspark.ml.classification import LogisticRegression
lr1 = LogisticRegression(featuresCol = 'idf', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0   )
lrModel1 = lr1.fit(trainingData1 )
predictions1 = lrModel1.transform(testData1)
predictions1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|   [aaa, five, star]|(46803,[35,113,15...|  0.0|[2.00421324194879...|[0.90853388993045...|       0.0|
|[aaron, pilan, li...|(46803,[0,1,2,3,4...|  0.0|[2.15162365736807...|[0.92261930845848...|       0.0|
|[aaron, tucker, r...|(46803,[0,2,4,5,7...|  0.0|[2.39013965031372...|[0.94466693148352...|       0.0|
|[abandon, hope, j...|(46803,[0,3,4,5,6...|  0.0|[2.54824895952945...|[0.95197730120651...|       0.0|
|[abandon, hope, l...|(46803,[0,1,2,3,4...|  0.0|[2.02562496800121...|[0.89687958017802...|       0.0|
|[abbi, blake, tog...|(46803,[4,5,7,11,...|  0.0|[1.95691294134048...|[0.89949857781712...|       0.0|
|[abbi, gline, fab...|(46803,[35,113,79...|  0.0|[1.94411417052431...|[0.

In [84]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator1.evaluate(predictions1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8288820468300311

In [85]:
print("Logistic Regression model with TF-IDF")

print("Accuracy :" ,evaluator1.evaluate(predictions1,{evaluator1.metricName: "accuracy"}))
print("f1 :"       ,evaluator1.evaluate(predictions1,{evaluator1.metricName: "f1"}))
print("Precision :",evaluator1.evaluate(predictions1,{evaluator1.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator1.evaluate(predictions1,{evaluator1.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression model with TF-IDF
Accuracy : 0.8716029341029341
f1 : 0.8288820468300311
Precision : 0.8452056361557285
Recall : 0.8716029341029341

In [86]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'Hashingtf', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0 )
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|   [aaa, five, star]|(65536,[12710,167...|  0.0|[2.01032558100820...|[0.90939654936551...|       0.0|
|[aaron, pilan, li...|(65536,[716,1433,...|  0.0|[2.19629705092497...|[0.92953976516600...|       0.0|
|[aaron, tucker, r...|(65536,[44,139,17...|  0.0|[2.40622538879812...|[0.94595541639241...|       0.0|
|[abandon, hope, j...|(65536,[44,2517,3...|  0.0|[2.54387758835219...|[0.95173889716810...|       0.0|
|[abandon, hope, l...|(65536,[44,216,10...|  0.0|[2.04762645543777...|[0.90356610926070...|       0.0|
|[abbi, blake, tog...|(65536,[44,1760,4...|  0.0|[1.95459840551172...|[0.89952681330236...|       0.0|
|[abbi, gline, fab...|(65536,[12710,144...|  0.0|[1.96856586939648...|[0.

In [87]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8271584604683104

In [88]:
print("Logistic Regression model with Hashing TF")

print("Accuracy :" ,evaluator.evaluate(predictions,{evaluator.metricName: "accuracy"}))
print("f1 :"       ,evaluator.evaluate(predictions,{evaluator.metricName: "f1"}))
print("Precision :",evaluator.evaluate(predictions,{evaluator.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator.evaluate(predictions,{evaluator.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression model with Hashing TF
Accuracy : 0.8707596801346801
f1 : 0.8271584604683104
Precision : 0.8437480958333341
Recall : 0.8707596801346801

## Naive Bayes

In [89]:
# Naive Bayes model - Hashing
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol = 'Hashingtf', labelCol='label',modelType='multinomial')
nbModel = nb.fit(trainingData) 
nb_predictions = nbModel.transform(testData)
nb_predictions.show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|   [aaa, five, star]|(65536,[12710,167...|  0.0|[-25.696435332148...|[0.98761938125323...|       0.0|
|[aaron, pilan, li...|(65536,[716,1433,...|  0.0|[-690.15281210504...|[0.99999984203226...|       0.0|
|[aaron, tucker, r...|(65536,[44,139,17...|  0.0|[-298.07005091194...|[0.99968098399669...|       0.0|
|[abandon, hope, j...|(65536,[44,2517,3...|  0.0|[-745.55807936640...|[0.99999901973490...|       0.0|
|[abandon, hope, l...|(65536,[44,216,10...|  0.0|[-875.52398198830...|[0.99058883687748...|       0.0|
|[abbi, blake, tog...|(65536,[44,1760,4...|  0.0|[-184.81998161923...|[0.99565049244025...|       0.0|
|[abbi, gline, fab...|(65536,[12710,144...|  0.0|[-43.821767521848...|[0.

In [90]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_nb = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_nb.evaluate(nb_predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8666053869512577

In [ ]:
print("Naive Bayes model with Hashing TF")
print("Accuracy :" ,evaluator_nb.evaluate(nb_predictions,{evaluator_nb.metricName: "accuracy"}))
print("f1 :"       ,evaluator_nb.evaluate(nb_predictions,{evaluator_nb.metricName: "f1"}))
print("Precision :",evaluator_nb.evaluate(nb_predictions,{evaluator_nb.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_nb.evaluate(nb_predictions,{evaluator_nb.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Naive Bayes model with Hashing TF
Accuracy : 0.8595328282828283
f1 : 0.8666053869512577
Precision : 0.8755673100870511
Recall : 0.8595328282828283

In [ ]:
# Naive Bayes model - TF-IDF
from pyspark.ml.classification import NaiveBayes
nb1 = NaiveBayes(featuresCol = 'idf', labelCol='label',modelType='multinomial')
nbModel1 = nb1.fit(trainingData1) 
nb_predictions1 = nbModel1.transform(testData1)
nb_predictions1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|   [aaa, five, star]|(46803,[35,113,15...|  0.0|[-148.62937308576...|[0.99999934185369...|       0.0|
|[aaron, pilan, li...|(46803,[0,1,2,3,4...|  0.0|[-3057.7363371239...|[1.0,2.9594463863...|       0.0|
|[aaron, tucker, r...|(46803,[0,2,4,5,7...|  0.0|[-1260.1214786739...|[0.99999999591506...|       0.0|
|[abandon, hope, j...|(46803,[0,3,4,5,6...|  0.0|[-3014.4697122284...|[1.0,1.0072001098...|       0.0|
|[abandon, hope, l...|(46803,[0,1,2,3,4...|  0.0|[-3268.5772506667...|[0.05266277130041...|       1.0|
|[abbi, blake, tog...|(46803,[4,5,7,11,...|  0.0|[-578.22067516633...|[0.99998542655607...|       0.0|
|[abbi, gline, fab...|(46803,[35,113,79...|  0.0|[-248.94342784957...|[0.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_nb1 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_nb1.evaluate(nb_predictions1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8282583544048864

In [ ]:
print("Naive Bayes model with TF-IDF")

print("Accuracy :" ,evaluator_nb1.evaluate(nb_predictions1,{evaluator_nb1.metricName: "accuracy"}))
print("f1 :"       ,evaluator_nb1.evaluate(nb_predictions1,{evaluator_nb1.metricName: "f1"}))
print("Precision :",evaluator_nb1.evaluate(nb_predictions1,{evaluator_nb1.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_nb1.evaluate(nb_predictions1,{evaluator_nb1.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Naive Bayes model with TF-IDF
Accuracy : 0.8000390812890813
f1 : 0.8282583544048864
Precision : 0.8784361584517387
Recall : 0.8000390812890812

# Linear SVC 

In [ ]:
# Linear SVC model -Hashing TF 
from pyspark.ml.classification import OneVsRest,LinearSVC
lsvc = LinearSVC(featuresCol = 'Hashingtf', labelCol = 'label',maxIter=5)
onevsrest = OneVsRest(classifier=lsvc,featuresCol = 'Hashingtf', labelCol = 'label')
lsvcModel = onevsrest.fit(trainingData)
lsvc_predictions = lsvcModel.transform(testData)
lsvc_predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|   [aaa, five, star]|(65536,[12710,167...|  0.0|[1.46607749552191...|       0.0|
|[aaron, pilan, li...|(65536,[716,1433,...|  0.0|[2.58715080617521...|       0.0|
|[aaron, tucker, r...|(65536,[44,139,17...|  0.0|[3.05221700745961...|       0.0|
|[abandon, hope, j...|(65536,[44,2517,3...|  0.0|[3.18566273553675...|       0.0|
|[abandon, hope, l...|(65536,[44,216,10...|  0.0|[1.76445796424057...|       0.0|
|[abbi, blake, tog...|(65536,[44,1760,4...|  0.0|[1.87941216990167...|       0.0|
|[abbi, gline, fab...|(65536,[12710,144...|  0.0|[1.32185963920248...|       0.0|
|[abbi, zander, wr...|(65536,[4062,4114...|  0.0|[1.63614658263362...|       0.0|
|[abigail, darling...|(65536,[2,44,361,...|  1.0|[-2.0559034633602...|       1.0|
|[abigail, mean,

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_lsvc = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_lsvc.evaluate(lsvc_predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8711893265369006

In [ ]:
print("Linear SVC model with Hashing TF")
print("Accuracy :" ,evaluator_lsvc.evaluate(lsvc_predictions,{evaluator_lsvc.metricName: "accuracy"}))
print("f1 :"       ,evaluator_lsvc.evaluate(lsvc_predictions,{evaluator_lsvc.metricName: "f1"}))
print("Precision :",evaluator_lsvc.evaluate(lsvc_predictions,{evaluator_lsvc.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_lsvc.evaluate(lsvc_predictions,{evaluator_lsvc.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear SVC model with Hashing TF
Accuracy : 0.8865034271284271
f1 : 0.8711893265369006
Precision : 0.8659671522647858
Recall : 0.8865034271284271

In [ ]:
# Linear SVC model - TF-IDF
from pyspark.ml.classification import OneVsRest,LinearSVC
lsvc1 = LinearSVC(featuresCol = 'idf', labelCol = 'label',maxIter=5)
onevsrest1 = OneVsRest(classifier=lsvc1,featuresCol = 'idf', labelCol = 'label')
lsvcModel1 = onevsrest1.fit(trainingData1)
lsvc_predictions1 = lsvcModel1.transform(testData1)
lsvc_predictions1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|   [aaa, five, star]|(46803,[35,113,15...|  0.0|[1.44569060723490...|       0.0|
|[aaron, pilan, li...|(46803,[0,1,2,3,4...|  0.0|[2.46602855278043...|       0.0|
|[aaron, tucker, r...|(46803,[0,2,4,5,7...|  0.0|[2.92320421151173...|       0.0|
|[abandon, hope, j...|(46803,[0,3,4,5,6...|  0.0|[2.63617653913506...|       0.0|
|[abandon, hope, l...|(46803,[0,1,2,3,4...|  0.0|[1.46018138879798...|       0.0|
|[abbi, blake, tog...|(46803,[4,5,7,11,...|  0.0|[1.74357459413831...|       0.0|
|[abbi, gline, fab...|(46803,[35,113,79...|  0.0|[1.30489982321802...|       0.0|
|[abbi, zander, wr...|(46803,[1,3,4,6,1...|  0.0|[1.61254980954291...|       0.0|
|[abigail, darling...|(46803,[0,1,2,5,6...|  1.0|[-2.2448775920825...|       1.0|
|[abigail, mean,

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_lsvc1 = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_lsvc1.evaluate(lsvc_predictions1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8713975696613668

In [ ]:
print("Linear SVC model with TF-IDF")
print("Accuracy :" ,evaluator_lsvc1.evaluate(lsvc_predictions1,{evaluator_lsvc1.metricName: "accuracy"}))
print("f1 :"       ,evaluator_lsvc1.evaluate(lsvc_predictions1,{evaluator_lsvc1.metricName: "f1"}))
print("Precision :",evaluator_lsvc1.evaluate(lsvc_predictions1,{evaluator_lsvc1.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_lsvc1.evaluate(lsvc_predictions1,{evaluator_lsvc1.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear SVC model with TF-IDF
Accuracy : 0.8878757816257816
f1 : 0.8713975696613668
Precision : 0.8675509250675701
Recall : 0.8878757816257816

# Classification Models with Sampling

# Logistic Regression

In [101]:
from pyspark.ml.classification import LogisticRegression
lr1_s = LogisticRegression(featuresCol = 'idf', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0   )
lrModel1_s = lr1.fit(trainingData1_s )
predictions1_s = lrModel1_s.transform(testData1_s)
predictions1_s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|[abandon, thi, bo...|(20986,[0,1,2,3,4...|  2.0|[-3.6806662160138...|[3.54824154146157...|       2.0|
|[absolut, connect...|(20986,[0,1,2,4,1...|  2.0|[-0.7581190040391...|[0.12312040005389...|       2.0|
|[absolut, disappo...|(20986,[0,1,2,3,7...|  2.0|[-1.8310215676955...|[0.02249683466869...|       2.0|
|[absolut, love, a...|(20986,[0,4,6,10,...|  0.0|[1.46265028998125...|[0.81000172496429...|       0.0|
|[absolut, love, b...|(20986,[1,2,4,6,7...|  0.0|[1.04175665259980...|[0.70002192045034...|       0.0|
|[absolut, love, r...|(20986,[0,1,3,4,6...|  0.0|[1.85105872520227...|[0.87904399572420...|       0.0|
|[absolut, love, r...|(20986,[2,4,6,24,...|  0.0|[1.37775098519728...|[0.

In [102]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator1_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator1_s.evaluate(predictions1_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7444823816870154

In [103]:
print("Logistic Regression model with TF-IDF with Sampling")

print("Accuracy :" ,evaluator1_s.evaluate(predictions1_s,{evaluator1_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator1_s.evaluate(predictions1_s,{evaluator1_s.metricName: "f1"}))
print("Precision :",evaluator1_s.evaluate(predictions1_s,{evaluator1_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator1_s.evaluate(predictions1_s,{evaluator1_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression model with TF-IDF with Sampling
Accuracy : 0.7498422522063082
f1 : 0.749235861172328
Precision : 0.7472344276357595
Recall : 0.7461716616543952

In [104]:
from pyspark.ml.classification import LogisticRegression
lr_s = LogisticRegression(featuresCol = 'Hashingtf', labelCol = 'label',maxIter=20, regParam=0.3, elasticNetParam=0 )
lrModel_s = lr_s.fit(trainingData_s)
predictions_s = lrModel_s.transform(testData_s)
predictions_s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|[abigail, conner,...|(65536,[892,3191,...|  1.0|[0.22849281707891...|[0.24603735215315...|       1.0|
|[absolut, awesom,...|(65536,[1542,2408...|  1.0|[-0.3737211056638...|[0.12480650768473...|       1.0|
|[absolut, connect...|(65536,[44,1696,2...|  2.0|[-0.0067519201578...|[0.24794033757148...|       2.0|
|[absolut, hate, w...|(65536,[7173,8371...|  2.0|[-0.1980434339323...|[0.27081626708069...|       1.0|
|[absolut, love, a...|(65536,[44,8371,8...|  1.0|[-0.3697418216559...|[0.19674311245430...|       1.0|
|[absolut, love, m...|(65536,[2517,5765...|  1.0|[-0.4441312073552...|[0.12696230385767...|       1.0|
|[absolut, love, p...|(65536,[6783,8371...|  1.0|[-0.3285348345430...|[0.

In [105]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_s.evaluate(predictions_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7516890853720156

In [106]:
print("Logistic Regression model with Hashing TF with Sampling")

print("Accuracy :" ,evaluator_s.evaluate(predictions_s,{evaluator_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator_s.evaluate(predictions_s,{evaluator_s.metricName: "f1"}))
print("Precision :",evaluator_s.evaluate(predictions_s,{evaluator_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_s.evaluate(predictions_s,{evaluator_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression model with Hashing TF with Sampling
Accuracy : 0.7531571218795888
f1 : 0.7494865077217701
Precision : 0.7516564221668667
Recall : 0.7517445676581338

# Naive Bayes

In [107]:
# Naive Bayes model - Hashing
from pyspark.ml.classification import NaiveBayes
nb_s = NaiveBayes(featuresCol = 'Hashingtf', labelCol='label',modelType='multinomial')
nbModel_s = nb_s.fit(trainingData_s) 
nb_predictions_s = nbModel_s.transform(testData_s)
nb_predictions_s.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|[abbi, love, joe,...|(65536,[1163,1198...|  1.0|[-335.15310319054...|[0.01322757532015...|       1.0|
|[absolut, disappo...|(65536,[44,716,26...|  2.0|[-453.80446645713...|[1.66585881642897...|       2.0|
|[absolut, fall, l...|(65536,[12,44,106...|  1.0|[-1649.2835003607...|[6.03361004898656...|       1.0|
|[absolut, hate, w...|(65536,[7173,8371...|  2.0|[-35.587062380394...|[0.06328718188687...|       1.0|
|[absolut, highli,...|(65536,[44,484,26...|  1.0|[-333.98917466774...|[6.57728148916824...|       1.0|
|[absolut, love, b...|(65536,[44,2692,4...|  1.0|[-143.99138453714...|[0.00166611874911...|       1.0|
|[absolut, love, b...|(65536,[44,216,40...|  1.0|[-465.01555246250...|[0.

In [108]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_nb_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_nb_s.evaluate(nb_predictions_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7188113870761607

In [109]:
print("Naive Bayes model with Hashing TF - with Sampling")
print("Accuracy :" ,evaluator_nb_s.evaluate(nb_predictions_s,{evaluator_nb_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator_nb_s.evaluate(nb_predictions_s,{evaluator_nb_s.metricName: "f1"}))
print("Precision :",evaluator_nb_s.evaluate(nb_predictions_s,{evaluator_nb_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_nb_s.evaluate(nb_predictions_s,{evaluator_nb_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Naive Bayes model with Hashing TF - with Sampling
Accuracy : 0.717037575003026
f1 : 0.7164522311737662
Precision : 0.7190989384603774
Recall : 0.7179400904165373

In [110]:
# Naive Bayes model - TF-IDF
from pyspark.ml.classification import NaiveBayes
nb1_s = NaiveBayes(featuresCol = 'idf', labelCol='label',modelType='multinomial')
nbModel1_s = nb1_s.fit(trainingData1_s) 
nb_predictions1_s = nbModel1_s.transform(testData1_s)
nb_predictions1_s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+----------+
|[abandon, thi, bo...|(20986,[0,1,2,3,4...|  2.0|[-12109.751088070...|[2.78252439064165...|       2.0|
|[absolut, ador, f...|(20986,[0,1,2,5,6...|  2.0|[-1893.1834163358...|[1.65715653128254...|       1.0|
|[absolut, ador, f...|(20986,[0,3,4,5,1...|  0.0|[-910.49271094868...|[0.99999999999921...|       0.0|
|[absolut, big, pu...|(20986,[0,1,2,3,5...|  0.0|[-6809.6681249137...|[1.0,6.5924307517...|       0.0|
|[absolut, cannot,...|(20986,[0,4,15,33...|  0.0|[-114.33041657309...|[0.99978001684327...|       0.0|
|[absolut, love, a...|(20986,[0,1,2,4,5...|  0.0|[-2453.4511345811...|[1.0,1.7778168520...|       0.0|
|[absolut, love, b...|(20986,[0,1,2,3,6...|  1.0|[-2379.5145151822...|[5.

In [111]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_nb1_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_nb1_s.evaluate(nb_predictions1_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6938176763531975

In [112]:
print("Naive Bayes model with TF-IDF")

print("Accuracy :" ,evaluator_nb1_s.evaluate(nb_predictions1_s,{evaluator_nb1_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator_nb1_s.evaluate(nb_predictions1_s,{evaluator_nb1_s.metricName: "f1"}))
print("Precision :",evaluator_nb1_s.evaluate(nb_predictions1_s,{evaluator_nb1_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_nb1_s.evaluate(nb_predictions1_s,{evaluator_nb1_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Naive Bayes model with TF-IDF
Accuracy : 0.6936067968297595
f1 : 0.6926962742744494
Precision : 0.6952295628668833
Recall : 0.6966014596018483

# Linear SVC 

In [113]:
# Linear SVC model -Hashing TF 
from pyspark.ml.classification import OneVsRest,LinearSVC
lsvc_s = LinearSVC(featuresCol = 'Hashingtf', labelCol = 'label',maxIter=5)
onevsrest_s = OneVsRest(classifier=lsvc_s,featuresCol = 'Hashingtf', labelCol = 'label')
lsvcModel_s = onevsrest_s.fit(trainingData_s)
lsvc_predictions_s = lsvcModel_s.transform(testData_s)
lsvc_predictions_s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+----------+
|         cleanTokens|           Hashingtf|label|       rawPrediction|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|[abandon, thi, bo...|(65536,[12,44,672...|  2.0|[-4.0772093039551...|       2.0|
|[absolut, ador, f...|(65536,[88,1433,1...|  2.0|[1.20614482435136...|       0.0|
|[absolut, ador, t...|(65536,[6425,8371...|  1.0|[-1.5571168147768...|       1.0|
|[absolut, disappo...|(65536,[44,716,26...|  2.0|[-0.7679124333021...|       2.0|
|[absolut, enjoi, ...|(65536,[3628,3920...|  1.0|[-0.7090226716535...|       1.0|
|[absolut, love, c...|(65536,[8371,1271...|  1.0|[-1.2983765107856...|       1.0|
|[absolut, love, c...|(65536,[5381,8371...|  0.0|[-1.0909432467226...|       1.0|
|[absolut, love, s...|(65536,[1303,2495...|  1.0|[-1.2748021265116...|       1.0|
|[absolut, love, t...|(65536,[1198,2206...|  1.0|[-2.5523205327544...|       1.0|
|[absolut, love,

In [114]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_lsvc_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_lsvc_s.evaluate(lsvc_predictions_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7433128052117389

In [115]:
print("Linear SVC model with Hashing TF")
print("Accuracy :" ,evaluator_lsvc_s.evaluate(lsvc_predictions_s,{evaluator_lsvc_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator_lsvc_s.evaluate(lsvc_predictions_s,{evaluator_lsvc_s.metricName: "f1"}))
print("Precision :",evaluator_lsvc_s.evaluate(lsvc_predictions_s,{evaluator_lsvc_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_lsvc_s.evaluate(lsvc_predictions_s,{evaluator_lsvc_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear SVC model with Hashing TF
Accuracy : 0.7451621271076524
f1 : 0.7433493235274405
Precision : 0.7404572988330357
Recall : 0.7453952997916973

In [116]:
# Linear SVC model - TF-IDF
from pyspark.ml.classification import OneVsRest,LinearSVC
lsvc1_s = LinearSVC(featuresCol = 'idf', labelCol = 'label',maxIter=5)
onevsrest1_s = OneVsRest(classifier=lsvc1_s,featuresCol = 'idf', labelCol = 'label')
lsvcModel1_s = onevsrest1_s.fit(trainingData1_s)
lsvc_predictions1_s = lsvcModel1_s.transform(testData1_s)
lsvc_predictions1_s.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+--------------------+----------+
|         cleanTokens|                 idf|label|       rawPrediction|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|[abandon, thi, bo...|(20986,[0,1,2,3,4...|  2.0|[-6.2293156418395...|       2.0|
|[absolut, ador, f...|(20986,[0,1,2,5,6...|  2.0|[-4.1656602064260...|       1.0|
|[absolut, ador, t...|(20986,[0,1,4,6,9...|  0.0|[1.22073856568915...|       0.0|
|[absolut, cannot,...|(20986,[0,4,15,33...|  0.0|[0.68020632299533...|       0.0|
|[absolut, connect...|(20986,[0,1,2,4,1...|  2.0|[-1.7902152255116...|       2.0|
|[absolut, love, b...|(20986,[0,1,2,3,6...|  1.0|[-4.5086593626763...|       1.0|
|[absolut, love, b...|(20986,[1,4,6,12,...|  0.0|[1.52147370183550...|       0.0|
|[absolut, love, k...|(20986,[3,6,18,72...|  0.0|[2.05755460736725...|       0.0|
|[absolut, love, t...|(20986,[0,1,4,6,8...|  1.0|[-0.2192580035289...|       2.0|
|[absolut, love,

In [117]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator_lsvc1_s = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator_lsvc1_s.evaluate(lsvc_predictions1_s)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.739857749996891

In [118]:
print("Linear SVC model with TF-IDF")
print("Accuracy :" ,evaluator_lsvc1_s.evaluate(lsvc_predictions1_s,{evaluator_lsvc1_s.metricName: "accuracy"}))
print("f1 :"       ,evaluator_lsvc1_s.evaluate(lsvc_predictions1_s,{evaluator_lsvc1_s.metricName: "f1"}))
print("Precision :",evaluator_lsvc1_s.evaluate(lsvc_predictions1_s,{evaluator_lsvc1_s.metricName: "weightedPrecision"}))
print("Recall :"   ,evaluator_lsvc1_s.evaluate(lsvc_predictions1_s,{evaluator_lsvc1_s.metricName: "weightedRecall"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear SVC model with TF-IDF
Accuracy : 0.7422198003701974
f1 : 0.7406096208001899
Precision : 0.738021494637489
Recall : 0.7414235705950991